<a href="https://colab.research.google.com/github/levizwannah/ml_tasks/blob/main/ML_Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV
from google.colab import drive


# **Cleaning and Preparing The data set**

In [ ]:
#mounting google drive to get the dataset
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#fetch the dataset from the drive
data=pd.read_csv('/content/drive/MyDrive/Machine Learning/modified_data.csv')

In [ ]:
#Preview the Dataset
data.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500


In [ ]:
#Shape of the dataset
data.shape

(1460, 77)

In [ ]:
#Length of Dataset
len(data)

1460

In [ ]:
#Dropping the ID column (Not needed)
data.drop('Id',inplace=True,axis=1)

In [ ]:
#Setting Original featured 
crude_features=list(data.columns)
crude_features

In [ ]:
#Get features with missing values 
features_missing_values=list(data.columns[data.isna().any()])
features_missing_values

['LotFrontage',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

# **Filling in Missing Features**

In [ ]:
#bfill and mean_fill to fill depending on respective data types
bfill,mean_fill=[],[]

In [110]:
for feature in features_missing_values:
  print(feature,' ',data[feature].isna().sum()/len(data)*100,'% ',data[feature].dtype)
  #stratify the missing features dependednt on their data types
  if data[feature].dtype=='float64':
    mean_fill.append(feature)
  else:
    bfill.append(feature)

LotFrontage   17.73972602739726 %  float64
MasVnrType   0.547945205479452 %  object
MasVnrArea   0.547945205479452 %  float64
BsmtQual   2.5342465753424657 %  object
BsmtCond   2.5342465753424657 %  object
BsmtExposure   2.6027397260273974 %  object
BsmtFinType1   2.5342465753424657 %  object
BsmtFinType2   2.6027397260273974 %  object
Electrical   0.0684931506849315 %  object
FireplaceQu   47.26027397260274 %  object
GarageType   5.5479452054794525 %  object
GarageYrBlt   5.5479452054794525 %  float64
GarageFinish   5.5479452054794525 %  object
GarageQual   5.5479452054794525 %  object
GarageCond   5.5479452054794525 %  object


In [ ]:
mean_fill

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [ ]:
bfill

['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [111]:
for feature in mean_fill:
  data[feature].fillna(data[feature].mean(),inplace=True)

In [ ]:
for feature in bfill:
  data[feature].fillna(method='bfill',inplace=True)

In [ ]:
#Confirm any missing value
data.columns[data.isna().any()]

Index([], dtype='object')

In [ ]:
data['FireplaceQu'].isna().sum()

0

In [ ]:
data['FireplaceQu'].fillna(method='ffill',inplace=True)

# ***Encoding the Dataset***

In [ ]:
#pick categorical features from the dataset using select_dtypes()
categorical_features=list(data.select_dtypes(include=['object']).columns)
categorical_features

['MSZoning', 'LandContour', 'Neighborhood']

In [ ]:
#pick numerical features from the dataset by taking the set difference between the crude features and the categorical features
numerical=list(set(crude_features)-set(categorical_features))
numerical


In [ ]:
categorical_features

['MSZoning', 'LandContour', 'Neighborhood']

In [ ]:
#Categorical features can be classified into two categories: Nominal and Ordinal features
nominal=['MSZoning','LandContour','Neighborhood']
ordinal=list(set(categorical_features)-set(nominal))

In [ ]:
#Let's take a moment to look at the whole ordinal data we have
data[ordinal]


In [ ]:
#Let's take a moment to look at the whole nominal data we have
data[nominal]


In [ ]:
#View target features
target=['SalePrice']

In [ ]:
#Since we will be doing computations, we convert categorical data to numerical

df_nominal=pd.get_dummies(data[nominal])
df_nominal.head(3)

In [ ]:
for feature in ordinal:
  data[feature]=data[feature].astype('category').cat.codes

df_ordinal=data[ordinal]

In [ ]:
#converted ordinal data
df_ordinal

In [ ]:
#our numerical data
data[numerical]

In [ ]:
new_data=pd.concat([df_nominal,df_ordinal,data[numerical]],axis=1)

In [ ]:
#Check if the shape of our dataset has remained intact 
new_data.shape

(1460, 107)

In [ ]:
#Encoded Data
new_data

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Neighborhood_Blmngtn,...,Electrical,OpenPorchSF,LotShape,GarageArea,FireplaceQu,MasVnrArea,MoSold,Condition1,YrSold,TotalBsmtSF
0,0,0,0,1,0,0,0,0,1,0,...,4,61,3,548,4,196.0,2,2,2008,856
1,0,0,0,1,0,0,0,0,1,0,...,4,0,3,460,4,0.0,5,1,2007,1262
2,0,0,0,1,0,0,0,0,1,0,...,4,42,0,608,4,162.0,9,2,2008,920
3,0,0,0,1,0,0,0,0,1,0,...,4,35,0,642,2,0.0,2,2,2006,756
4,0,0,0,1,0,0,0,0,1,0,...,4,84,0,836,4,350.0,12,2,2008,1145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,1,0,0,0,0,1,0,...,4,40,3,460,4,0.0,8,2,2007,953
1456,0,0,0,1,0,0,0,0,1,0,...,4,0,3,500,4,119.0,2,2,2010,1542
1457,0,0,0,1,0,0,0,0,1,0,...,4,60,3,252,2,0.0,5,2,2010,1152
1458,0,0,0,1,0,0,0,0,1,0,...,0,0,3,240,2,0.0,4,2,2010,1078


# **Standardizing the Dataset**

In [ ]:
#Drop the target feature
new_data.drop('SalePrice',axis=1,inplace=True)

In [ ]:
#Check the new dataset
new_data

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Neighborhood_Blmngtn,...,BsmtFinSF2,GarageArea,YearBuilt,MasVnrArea,FullBath,YearRemodAdd,MoSold,TotRmsAbvGrd,YrSold,TotalBsmtSF
0,0,0,0,1,0,0,0,0,1,0,...,0,548,2003,196.0,2,2003,2,8,2008,856
1,0,0,0,1,0,0,0,0,1,0,...,0,460,1976,0.0,2,1976,5,6,2007,1262
2,0,0,0,1,0,0,0,0,1,0,...,0,608,2001,162.0,2,2002,9,6,2008,920
3,0,0,0,1,0,0,0,0,1,0,...,0,642,1915,0.0,1,1970,2,7,2006,756
4,0,0,0,1,0,0,0,0,1,0,...,0,836,2000,350.0,2,2000,12,9,2008,1145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,1,0,0,0,0,1,0,...,0,460,1999,0.0,2,2000,8,7,2007,953
1456,0,0,0,1,0,0,0,0,1,0,...,163,500,1978,119.0,2,1988,2,7,2010,1542
1457,0,0,0,1,0,0,0,0,1,0,...,0,252,1941,0.0,2,2006,5,9,2010,1152
1458,0,0,0,1,0,0,0,0,1,0,...,1029,240,1950,0.0,1,1996,4,5,2010,1078


In [ ]:
numpy_array = new_data.to_numpy()
numpy_array

array([[   0.,    0.,    0., ...,    8., 2008.,  856.],
       [   0.,    0.,    0., ...,    6., 2007., 1262.],
       [   0.,    0.,    0., ...,    6., 2008.,  920.],
       ...,
       [   0.,    0.,    0., ...,    9., 2010., 1152.],
       [   0.,    0.,    0., ...,    5., 2010., 1078.],
       [   0.,    0.,    0., ...,    6., 2008., 1256.]])

In [ ]:
numpy_array[1]

In [ ]:
#Standardizing the dataset
standardized_data=StandardScaler().fit_transform(numpy_array)

In [ ]:
#smaple of the standerdized dataset
standardized_data[0]

In [ ]:
#Target feature represents the Y variable in the Y=mX+b equation
target_feature=data[target]
target_feature

In [ ]:
#we convert the target feature to numpy array using to_numpy()
target_feature=target_feature.to_numpy()

In [ ]:
target_feature.shape

(1460, 1)

In [ ]:
#A peek of how our target_feature looks like in numpy array's format
target_feature

In [ ]:
#Standerdized the target feature using StrandardScaler()
#fit and transform the target feature using fit_transform()
target_feature=StandardScaler().fit_transform(target_feature)

In [ ]:
#A peek of how our target_feature looks like after standerdization
target_feature

In [ ]:
standardized_data.shape

(1460, 106)

# ***Extracting Feature Using PCA***

In [ ]:
#specifying the number of components we are extracting
pca=PCA(n_components=4)

In [ ]:
#fit and transform the target feature using fit_transform()
p_components=pca.fit_transform(standardized_data)

In [ ]:
#peek of the components extracted 
p_components

array([[ 2.4433097 , -1.27896883, -1.03947519, -1.41896811],
       [ 0.31645543,  2.63975128, -1.23374282, -0.62225771],
       [ 2.85722029, -0.81104242, -0.88052826, -1.16898726],
       ...,
       [ 1.26182064, -0.85886434,  2.77459822, -1.49861468],
       [-2.85096273,  3.29384871, -1.45669888,  0.44760798],
       [-0.77738539,  2.79013395, -1.36336903, -0.8580012 ]])

In [ ]:
#explained_variance_ratio_ method of PCA is used to get the ratio of variance (eigenvalue / total eigenvalues)
pca.explained_variance_ratio_

array([0.10560233, 0.04472091, 0.04181921, 0.03216466])

# Doing Feature Selection

In [ ]:
#get regressor to find the L1 Coefficients for the columns in the PCS
regressor = LassoCV();
pca_data = pd.DataFrame(p_components)
pca_data
regressor.fit(pca_data, np.ravel(target_feature))

LassoCV()

In [ ]:
# Get the regressor cofficients for each column
coef  = pd.Series(regressor.coef_, index = pca_data.columns)
num_of_selected_features = sum(coef != 0)
num_of_selected_features


4

# New Dataset Ready for Training


# Train the Model

In [ ]:
#LassoCV is the Lasso Regression model to have a more accurate prediction
#CV is cross-validation and the default fold is 5
reg=LassoCV()

In [ ]:
#get pca with 2 components
pca = PCA(n_components = 2)
p_components = pca.fit_transform(standardized_data)
pca_data = pd.DataFrame(p_components)

In [ ]:
#reshape target feature
#r_target_feature = np.reshape(target_feature, (-1, 1))
reg.fit(pca_data, np.ravel(target_feature))

LassoCV()

In [ ]:
#
reg.coef_

array([0.25387739, 0.01887053])